## LP Workout Manager
author = ['Matt Guan']

### Inputs 

In [1]:
from io import StringIO
import sqlite3

import numpy as np
import pandas as pd

from workout_maker import WorkoutMaker
from functions.db_funcs import get_db_con, create_db

__Create Tables if they Don't Exist__

In [2]:
con = get_db_con(db='workout.db')
create_db(con)

In [3]:
import datetime
from functions.dt_funcs import now
import json

__initialize module__

In [4]:
workout = WorkoutMaker(con, 'Matt Guan', 'email@email.com')

Welcome back Matt Guan!


### __First Run Only__
Enter your workout inputs here..
* `accessory` : pandas df containing accessory workout weights
* `prog` : dict containing monthly progression weight for each main workout
* `orm` : dict containing one rep max weights for each main workout

<div class="alert alert-info">
  <strong>Note:</strong> You only need to run this part if it is your first run OR you want to update something.
</div>

In [5]:
s = '''
me_name	ae_name	ae_weight	sets	reps
deadlift	Rows	100	4	10
deadlift	Pullups	0	4	10
deadlift	Curls	30	4	10
deadlift	Back Extension	225	4	10
squat	Leg Press	290	4	10
squat	Lying Leg Curl	85	4	10
squat	Calf Raise	225	4	10
squat	Hip Abd/Add-uction	250	4	10
bench	Dips	0	4	10
bench	Incline D.Bell Press	45	4	10
bench	Cable Tricep Ext.	42.5	4	10
bench	Machine Pec Fly	100	4	10
ohp	Sitting D.Bell Press	50	5	5
ohp	Cable Rear Delt Fly	25	4	10
ohp	Machine Rear Delt Fly	70	4	10
ohp	Machine Lat Raise	60	4	10
'''
accessory = pd.read_csv(StringIO(s), sep='\t')

prog = {'squat':2.5 , 'deadlift':2.5, 'ohp':0, 'bench':2.5}
orm = {'squat':240 , 'deadlift':290, 'ohp':127.5, 'bench':182.5}

In [6]:
workout.set_accessory(accessory)

2018-06-17 16:34:11,154 | INFO | dataframe is valid - accessory populated


,me_name,ae_name,ae_weight,sets,reps,user_id,publish_time
0,deadlift,Rows,100.0,4,10,1,2018-06-17 16:34:11.144943
1,deadlift,Pullups,0.0,4,10,1,2018-06-17 16:34:11.144943
2,deadlift,Curls,30.0,4,10,1,2018-06-17 16:34:11.144943


In [7]:
workout.set_dim_prog(prog)

2018-06-17 16:34:11,419 | INFO | duplicate entry deleted
2018-06-17 16:34:11,425 | INFO | new entry loaded
2018-06-17 16:34:11,438 | INFO | dict is valid - dim_progression populated


,user_id,prog_dict
0,1,"{""squat"": 2.5, ""deadlift"": 2.5, ""ohp"": 0, ""ben..."


In [8]:
workout.set_one_rep_max(orm)

2018-06-17 16:34:11,990 | INFO | duplicate entry deleted
2018-06-17 16:34:11,995 | INFO | new entry loaded
2018-06-17 16:34:12,010 | INFO | dict is valid - one_rep_max overwitten


### Pull Workout

In [9]:
workout.main()

2018-06-17 16:34:13,737 | INFO | using current orm - use self.set_one_rep_max if you wish to modify it
2018-06-17 16:34:13,839 | INFO | workout saved to ./lp-workout.html


### Extras

__Check out some of the intermediate Output__

In [10]:
workout.get_accessory().head(3)

,me_name,ae_name,ae_weight,sets,reps
0,deadlift,Rows,100.0,4,10
1,deadlift,Pullups,0.0,4,10
2,deadlift,Curls,30.0,4,10


In [11]:
orm = workout.get_orm()
orm

,user_id,data_start_date,data_end_date,orm_dict,publish_time
0,1,2018-06-17,2018-07-14,"{""squat"": 240, ""deadlift"": 290, ""ohp"": 127.5, ...",2018-06-17 16:26:44.019871


In [12]:
from functions.workout_funcs import get_workout
from functions.db_funcs import retrieve_json

In [13]:
orm_dict = retrieve_json(orm, 'orm_dict')
orm_dict

{'bench': 182.5, 'deadlift': 290, 'ohp': 127.5, 'squat': 240}

In [14]:
get_workout(orm_dict, [1])

,week,set,reps,deadlift,squat,bench,ohp
0,1,1,5,190,155,120,85
1,1,2,5,220,180,135,95
2,1,3,5,245,205,155,110


__Check out the Database__

In [15]:
schema = pd.read_sql('SELECT name FROM sqlite_master WHERE type="table";', con)
schema

,name
0,dim_user
1,dim_progression
2,one_rep_max
3,accessory
4,pause_workout


In [16]:
pd.read_sql('select * from dim_user', con)

,user_id,user_name,email
0,1,Matt Guan,email@email.com


In [17]:
pd.read_sql('select * from one_rep_max', con)

,user_id,data_start_date,data_end_date,orm_dict,publish_time
0,1,2018-06-10,2018-06-16,"{""squat"": 240, ""deadlift"": 290, ""ohp"": 127.5, ...",2018-06-17 16:26:44.049850
1,1,2018-06-17,2018-07-14,"{""squat"": 240, ""deadlift"": 290, ""ohp"": 127.5, ...",2018-06-17 16:26:44.019871


In [18]:
pd.read_sql('select * from dim_progression', con)

,user_id,prog_dict
0,1,"{""squat"": 2.5, ""deadlift"": 2.5, ""ohp"": 0, ""ben..."
